# ketos jupyter notebook machine learning example

Hi there, this is your personal model building environment. You have full access to your environment and can install any R packages you need and build your own models.

Rhis is an example for a ketos jupyter notebook, that will guide you through the model development and deployment process.

TRAIN YOUR MODEL

To train your model you can install any R packages you might need, add any additional functions and build your own model.

For any good building process you will need data, here is how you get it:

1. go to your ketos user interface and request a new dataset. Once your dataset has been prepared for you, you can access it from within this notebook, to do this just substitute the string 
"paste your training data url from your ketos system here to get your training data"
in the "ketos_train" cell with the URL you got.


DEPLOY YOUR MODEL

to deploy the notebook in order to allow for external predictions just do the following:

1. tag the cells that you would like to be executed for your prediction with the keyword "ketos_predict" in the first line of each cell as a comment.

2. tag the cells which produce the output for the prediction with the keyword "ketos_predict_output", please note that this cell will be executed and the output will be used as return value for the prediction. It is expected to be in json format as supplied by the "format_to_json" function provided here.


# please note that if you would like to test a prediction using this model on our demo server, you will need to call one of the following patient ids in the front end: https://ketos.ai/mlmodels

Requirement for patientid:

1 <= patientid <= 140

In [ ]:
# ketos_init
install.packages(c('caret', 'plyr', 'e1071', 'glmnet'), dependencies=TRUE)

In [ ]:
# ketos_train
genPrep = function(){  
  library(caret)
}

splitData = function(seed, data){
  ## randomly splits data into training (0.8) and test (0.2) data
  set.seed(seed)
  training_index = createDataPartition(data[[ncol(data)]], p = 0.8, list = F)
  training = data[training_index,]
  test = data[-training_index,]
  
  return(list(training=training,test=test))
}

In [ ]:
# ketos_train
ketos_train = function(dataset = NULL, outcomeVar = NULL, metric = "Accuracy", control = NULL){
  # trains model - in this example a glmnet model
  form = as.formula(paste(c(outcomeVar, "~."), collapse=" "))
  
  print("start train GLMNET")
  set.seed(seed)
  trained_model = caret::train(form, data=dataset, method="glmnet", metric=metric, preProc=c("center", "scale"), trControl=control)

  return(trained_model)
  
}

In [ ]:
# ketos_train
ketos_Input_data = ("http://ketos_preproc:5000/aggregation/5d7f46e8239b696559b819a1?output_type=csv&aggregation_type=latest")
ketos_data = read.csv(url(ketos_Input_data))
genPrep()
seed = 7

splitData <- splitData(seed, ketos_data)
trainingData<-splitData$training
testData <- splitData$test
write.table(trainingData, 'training_data.csv', sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)
write.table(testData, 'test_data.csv', sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)
data <- read.csv("training_data.csv", header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
data

control = trainControl(method="repeatedcv", number=10, repeats=10, sampling="down", savePredictions="all")
outcomeVar = "bodyComposition"
ketos_model = ketos_train(trainingData, outcomeVar,"Accuracy",control)
modeltoSave = 'ketos_model'
save(ketos_model,file = modeltoSave)

In [ ]:
#ketos_predict, ketos_predict_output

format_to_json = function(patientIds, predictions){
    ret_json = "["
    ret_json = paste(ret_json,'{"patientId":"',patientIds[1] ,'", "prediction":"',predictions[1], '"}', sep="")
    
    if(!length(patientIds) == 1){
    for(i in 2:length(patientIds)){
        patientId = patientIds[i]
        prediction = predictions[i]
        
        ret_json = paste(ret_json,',{"patientId":"',patientId ,'", "prediction":"' ,prediction, '"}', sep="")
    }
        }
    ret_json = paste(ret_json, "]", sep="")
    
    return (ret_json)
}

ketos_predict = function(){
    ketos_predict_data = ("csv url will be inserted for the predict call here automatically - paste your url here to test your predict function")
    load("ketos_model")
    predict_data = read.csv(url(ketos_predict_data)) 
    prediction = predict(ketos_model, predict_data)
    prediction = format_to_json(predict_data$patient, prediction)
    return (prediction)
}
print(ketos_predict(), quote=FALSE)